# Every Variant Sequencing with Oxford Nanopore Technologies

This script is being used after sequencing. The raw pod5 files can be basecalled or the already basecalled files can be used directly (fastq.gz)

## Workflow

### 1. Basecalling (Optional)

- The raw reads are stored in the main folder of ONT (e.g /var/lib/minknow/data). Enter the experiment name as input. 
- Sequences are basecalled based on the model of choice. If enough computational power is available, we recommend "sup" method

### 2. Demultiplexing 
- Each reead is assigned to a well/plate combination. 

### 3. Variant Calling
- Minimap2 for creating Multiple Sequence Alignment (MSA)
- Base Frequency Caller is being used for variant calling



### Packages 

In [15]:
# Import all packages

import sys
sys.path.append("/home/emre/github_repo/MinION")

from minION.util import IO_processor
from minION.basecaller import Basecaller

from minION.variantcaller import *

from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import subprocess
import importlib
importlib.reload(IO_processor)

<module 'minION.util.IO_processor' from '/home/emre/github_repo/MinION/minION/util/IO_processor.py'>

### Meta Data 

- Provide the following arguments:

- Result Path: Path where the minion result folder will be created. All experiment results are then stored within the folder
- Experiment Name: The experiment name is assigned when running the sequencer. Use the same name for identification


In [ ]:
result_path = Path("/home/emre/")
experiment_name = "20240206-MinION-KingBob10"
basecall_model_type = "sup"
result_folder = IO_processor.create_folder( experiment_name,
                                            basecall_model_type, 
                                            target_path=result_path)




# Create Barcode fasta file 
barcode_path = "../minION/barcoding/minion_barcodes.fasta" # Path to standard barcode file
front_prefix = "NB"
back_prefix = "RB"
bp = IO_processor.BarcodeProcessor(barcode_path, front_prefix, back_prefix)
barcode_path = result_folder / "minion_barcodes_filtered.fasta"

# Barcode indexes
front_min = 1
front_max = 96
back_min = 1
back_max = 12

# Expected fragment sizes
min_size = 800
max_size = 5000

bp.filter_barcodes(barcode_path, (front_min,front_max), (back_min,back_max))


file_to_experiment= f"/var/lib/minknow/data/{experiment_name}"
template_fasta = "/home/longy/zq-p411.fasta"

# Basecalling
basecall_folder = result_folder / "basecalled"
basecall_folder.mkdir(parents=True, exist_ok=True)
experiment_folder = IO_processor.find_experiment_folder(experiment_name) # Folder where pod5 files are located

# Demultiplexing
experiment_name = experiment_name + "_" + basecall_model_type
result_folder_path = IO_processor.find_folder(result_path, experiment_name)


In [ ]:
# Add conditions to avoid running the script accidentally
skip_basecalling = True
skip_demultiplex = False
skip_variant_calling = False

### Step 1 (Optional): Basecall reads

- Basecall can usually be done while sequencing (if GPU available?)
- Otherwise, basecall afterwards

In [ ]:
if not skip_basecalling:


    pod5_files = IO_processor.find_folder(experiment_folder, "pod5")
    bc = Basecaller(model=basecall_model_type, pod5_files, basecall_folder, fastq = True)
    bc.run_basecaller()


In [ ]:
# Find fastq files
file_to_fastq = IO_processor.find_folder(experiment_folder, "fastq_pass")
print(file_to_fastq)

### Step 2: Demultiplex with SW
- Demultiplex with SW 

In [ ]:
if not skip_demultiplex:
    path_to_code = "/home/emre/github_repo/MinION/source/source/demultiplex"
    prompt = f"{path_to_code} -f {file_to_fastq} -d {result_folder} -b {barcode_path} -w {100} -r {100} -m {min_size} -x {max_size}"
    subprocess.run(prompt, shell=True)

In [ ]:
demultiplex_folder = result_folder 
print(demultiplex_folder)

### Step 3: Call Variant with PileUP Analysis

- Call Variant with min freq of 0.4 & depth min 15

Read Summary file (Optional):


In [ ]:

demultiplex_folder_name = result_folder

In [16]:
if not skip_variant_calling:
    vc = VariantCaller(experiment_folder, 
                   template_fasta, 
                   demultiplex_folder_name=demultiplex_folder_name, 
                   padding_start=0, 
                   padding_end=0)
    
    variant_df = vc.get_variant_df(qualities=True, 
                                threshold=0.2,
                                min_depth=5)
    seq_gen = IO_processor.SequenceGenerator(variant_df, template_fasta)
    variant_df = seq_gen.get_sequences()
    #TODO: Save the variant_df to a file after running. Currently it is not saved.

0it [00:00, ?it/s]

unsupported operand type(s) for /: 'float' and 'str'


68it [00:39,  1.61it/s]

'NoneType' object is not subscriptable


71it [00:40,  1.62it/s]

...

'[1164, 1165, 1167, 1168] not in index'
'[2015, 2016, 2018, 2019] not in index'
Too many positions: 80, Skipping...
'[421, 422, 424, 425] not in index'
'[577, 578, 580, 581] not in index'
Too many positions: 35, Skipping...
'[1481, 1482, 1484, 1485] not in index'
Too many positions: 31, Skipping...
'[585, 586, 588, 589] not in index'


372it [03:03, 45.34it/s]

Too many positions: 73, Skipping...
'[591, 592, 594, 595] not in index'
Too many positions: 34, Skipping...
Too many positions: 28, Skipping...
'[290, 291, 293, 294] not in index'
'[1870, 1871, 1873, 1874] not in index'
Too many positions: 30, Skipping...
'[1486, 1487, 1489, 1490] not in index'


384it [03:03,  2.09it/s]

Too many positions: 40, Skipping...
'[1587, 1588, 1590, 1591] not in index'
'[1192, 1193, 1195, 1196] not in index'
'[140, 141, 143, 144] not in index'
'[21, 22, 24, 25] not in index'
'[161, 162, 164, 165] not in index'
Too many positions: 14, Skipping...
'[955, 956, 958, 959] not in index'
'[500, 501, 503, 504] not in index'
Too many positions: 35, Skipping...


In [ ]:
#20 - 30
variant_df.to_csv(result_folder / "variant_df.csv", index=False)  

In [ ]:
variant_df.iloc[1,2]

In [17]:
variant_df

,Plate,Well,Path,Alignment_count,Variant,Probability,Sequence
0,9,A1,NaN,0,NaN,NaN,NaN
1,9,A2,/home/emre/minION_results/20240206-MinION-King...,37,C541G_G543T,0.972916,ACCGTTATTAAACACAGATAAACCGGTTCAAGCTTTGATGAAAATT...
2,9,A3,/home/emre/minION_results/20240206-MinION-King...,49,T542G_G543T,0.959444,ACCGTTATTAAACACAGATAAACCGGTTCAAGCTTTGATGAAAATT...
3,9,A4,/home/emre/minION_results/20240206-MinION-King...,29,C541T_T542G,0.735540,ACCGTTATTAAACACAGATAAACCGGTTCAAGCTTTGATGAAAATT...
4,9,A5,/home/emre/minION_results/20240206-MinION-King...,65,C541G_T542G_G543T,1.000000,ACCGTTATTAAACACAGATAAACCGGTTCAAGCTTTGATGAAAATT...
...,...,...,...,...,...,...,...
379,12,H8,/home/emre/minION_results/20240206-MinION-King...,118,NaN,NaN,NaN
380,12,H9,/home/emre/minION_results/20240206-MinION-King...,267,nan,NaN,NaN
381,12,H10,/home/emre/minION_results/20240206-MinION-King...,249,NaN,NaN,NaN
382,12,H11,/home/emre/minION_results/20240206-MinION-King...,166,NaN,NaN,NaN
